# <a href="https://miptstats.github.io/courses/ad_fivt.html">Введение в анализ данных</a>
## Домашнее задание 10. Анализ вакансий.


**Правила, <font color="red">прочитайте внимательно</font>:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_ds24_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. Дождитесь подтверждения от бота, что он принял файл. Если подтверждения нет, то что-то не так. **Работы, присланные иным способом, не принимаются.**
* Дедлайн см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb` и полученные данные** (подробности далее). Если вы строите интерактивные графики, их стоит прислать в формате html.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Будьте внимательны при работе со сбором данных. Ответственность за корректность ваших действий лежит на вас. Не нагружайте сервера, *делайте паузы между запросами*. Как следствие, начинайте выполнять задание заранее. **Если вас где-то забаннили и т.п., то это не является уважительной причиной продления дедлайна.**
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания будут сдавать устный зачет.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*


---

**Баллы за задание:**

<b><a href="#Сложная-часть" style="text-decoration: none;"><font color="orange">Сложная часть</font></a></b> (учитывается только в основной части курса, необходимо на "отл"):
* Задача 1 &mdash; 70 баллов

<b><a href="#Факультативная-часть" style="text-decoration: none;"><font color="green">Факультативная часть</font></a></b> (учитывается только в факультативной части курса):
* Задача 2 &mdash; 30 баллов
* Задача 3 &mdash; 70 баллов

**Внимание!** Указанные баллы являются бонусными. Они не учитываются в максимальных суммах баллов в знаменателе при вычислении процента выполненных заданий. Но все полученные вами баллы будут учтены в числителе. Тем самым, выполняя или не выполняя домашку, вы не сможете понизить оценку, но сможете ее повысить.

In [ ]:
# Bot check

# HW_ID: fpmi_ad10
# Бот проверит этот ID и предупредит, если случайно сдать что-то не то.

# Status: not final
# Перед отправкой в финальном решении удали "not" в строчке выше.
# Так бот проверит, что ты отправляешь финальную версию, а не промежуточную.
# Никакие значения в этой ячейке не влияют на факт сдачи работы.

In [1]:
import requests
from time import sleep
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk

nltk.download("stopwords")

from string import punctuation

sns.set(style="whitegrid", font_scale=1.3, palette="Set2")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
stop_words = nltk.corpus.stopwords.words("english") + nltk.corpus.stopwords.words("russian")


Второй курс &mdash; самое время задуматься о будущей профессии и проанализировать существующие предложения. Дело тут даже не в том, чтобы найти интересную стажировку. В первую очередь сейчас стоит подумать о том, в какую сторону развиваться дальше. Например, если вы хотите работать в какой-либо конкретной профессии, то наверняка стоит развивать какие-то определенные навыки, и даже выбрать подходящую кафедру. Анализ существующих вакансий поможет определить, какие навыки вам нужны.

**Внимание!**
В первую очередь задание призвано помочь вам понять, какие навыки стоит развивать, и может быть даже выбрать подходящую кафедру. Однако, не стоит идти на работу/стажировку раньше времени. Даже только летнюю, абсолютное большинство людей не могут работать круглый год без перерыва. У всех из вас 3 курс достаточно нагруженный, а на DS-потоке нагрузка с первой недели сентября. И если вы идете на стажировку, то каникул у вас может просто не остаться. Даже если вам сейчас/летом/в сентябре кажется "я же справлюсь", к ноябрю-декабрю с большой вероятностью вы выдохнетесь. Таких примеров *очень много*. Не все, кто-то успешно справляется совмещать работу и учебу, но таких *очень мало*. Берегите себя и свое здоровье :)

---

Задание предполагает вариативность исследуемых данных. Вы можете выбрать один из следующих вариантов.
1. Стандартный вариант &mdash; проанализировать вакансии на сайте <a target="_blank" href="https://hh.ru/">hh.ru</a> с использованием <a target="_blank" href="https://dev.hh.ru/">официального API</a>. Далее приведен пример работы с этим API. *Если вы не достаточно уверены в своих силах, рекомендуется выбрать этот вариант.*
2. Разобраться с API других сервисов по поиску вакансий, например, <a target="_blank" href="https://career.habr.com/">Хабр.Карьера</a> или <a target="_blank" href="https://www.superjob.ru/">SuperJob</a>.
3. Извлечение данных с помощью <a target="_blank" href="https://miptstats.github.io/courses/ad_fivt/data_parsing.html#2.-Парсинг-HTML-напрямую">парсинга HTML напрямую</a> некоторых сайтов. Например, вы можете попробовать напрямую распарсить пул вакансий из тех источников, которые не предоставляют API. **Внимание! Обращайте внимание на пользовательские соглашения и на <a target="_blank" href="https://miptstats.github.io/courses/ad_fivt/data_parsing.html#Сбор-данных-из-открытых-источников">эти рекомендации</a>.**
4. Комбинирование нескольких подходов. В этом случае нужно понимать, как находить одинаковые вакансии из разных источников для избежания их двойного учета.
5. Можно также *дополнительно* брать архивные данные из недавнего прошлого, например, из <a target="_blank" href="https://boosters.pro/championship/HeadHunter/overview/description">этого соревнования</a>.

Перед выполнением задания ознакомьтесь с <a target="_blank" href="https://miptstats.github.io/courses/ad_fivt/data_parsing.html">материалом</a> о сборе данных из открытых источников.

Для получения максимального балла за задание достаточно выполнить все задание по стандартному варианту. Альтернативные варианты, даже более сложные, не дают больше баллов, чем максимальное количество баллов по стандартному варианту.

Работа в некоторой степени творческая, поэтому конкретные детали решения в основном остаются на усмотрение автора решения. Например, если в вакансии зарплата укзана "*от ...*" или  "*до ...*", то вы сами решаете, как это обрабатывать, но не забывайте про четкую аргументацию. Не забывайте также заглядывать в презентацию по выполнению и оформлению домашних заданий с занятия 2.

 ---
### Справка по работе с API сервиса hh.ru.   

<b><font color="red">Внимание!</font> При работе с API не забывайте делать паузы между запросами, чтобы не задудосить сервер. Если вас заблокируют, это не будет являться уважительной причиной переноса дедлайна.</b>

Мы будем работать только с вакансиями. Для этого не требуется регистрироваться и получать токен. Ниже приведен краткий пример работы с API. Подробное описание работы с вакансиями, включая параметры запросов и формат ответа можно почитать в <a target="_blank" href="https://github.com/hhru/api/blob/master/docs/vacancies.md#search">документации</a>.

Например, мы хотим найти вакансии по запросу `Data Scientist` в Москве. Тогда первую страницу поиска из 10 вакансий на страницу мы можем получить с запроса к API:

In [ ]:
URL = "https://api.hh.ru/vacancies"

params = {
    "text": "Data Scientist",
    "area": 1,
    "page": 0,
    "per_page": 10,
}

req = requests.get(URL, params)
data = json.loads(req.content.decode())

Если все прошло успешно, полученный словарь будет иметь следующие ключи

In [ ]:
data.keys()

dict_keys(['items', 'found', 'pages', 'page', 'per_page', 'clusters', 'arguments', 'fixes', 'suggests', 'alternate_url'])

Можем посмотреть на краткое описание первой вакансии

In [ ]:
data["items"][0]

{'id': '98612984',
 'premium': False,
 'name': 'Petroleum Engineer (Baku)',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2024-05-08T13:10:45+0300',
 'created_at': '2024-05-08T13:10:45+0300',
 'archived': False,
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=98612984',
 'show_logo_in_search': None,
 'insider_interview': None,
 'url': 'https://api.hh.ru/vacancies/98612984?host=hh.ru',
 'alternate_url': 'https://hh.ru/vacancy/98612984',
 'relations': [],
 'employer': {'id': '1068',
  'name': 'Cornerstone Russia',
  'url': 'https://api.hh.ru/employers/1068',
  'alternate_url': 'https://hh.ru/employer/1068',
  'logo_urls': {'240': 'https://img.hhcdn.ru/employer-logo/1643847.jpeg',
   'original': 'https://img

Сколько всего найдено вакансий

In [ ]:
data["found"]

359

Количество страниц в результатах поиска

In [ ]:
data["pages"]

36

Из результатов можем сделать удобную таблицу, причем в дальнейшем можно оставить только те колонки, которые необходимы для анализа.

In [ ]:
df = pd.json_normalize(data["items"])
df.head()

,id,premium,name,department,has_test,response_letter_required,salary,address,response_url,sort_point_distance,...,address.metro.line_name,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,branding.type,branding.tariff,employer.logo_urls,department.id,department.name
0,98612984,False,Petroleum Engineer (Baku),NaN,False,False,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,98594437,False,Data Scientist,NaN,False,False,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,97821551,False,Chief data officer,NaN,False,False,NaN,NaN,None,None,...,Замоскворецкая,2.2,2,55.706634,37.657008,MAKEUP,None,NaN,NaN,NaN
3,98592181,False,Аналитик данных/Junior DATA SCIENTIST,NaN,False,False,NaN,NaN,None,None,...,Филевская,4.73,4,55.740544,37.534100,MAKEUP,None,NaN,NaN,NaN
4,97615359,False,Junior Data Scientist / Младший аналитик данных,NaN,False,False,NaN,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,MAKEUP,None,NaN,NaN,NaN


Для получения полного описания вакансии потребуется задать отдельный запрос, используя ее `id`.

In [ ]:
vacancy = df["id"].iloc[0]
vacancy_url = f"https://api.hh.ru/vacancies/{vacancy}"

req = requests.get(vacancy_url)
vacancy_info = json.loads(req.content.decode())

In [ ]:
vacancy_info

{'id': '98612984',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Petroleum Engineer (Baku)',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'experience': {'id': 'between3And6', 'name': 'От 3 до 6 лет'},
 'schedule': {'id': 'fullDay', 'name': 'Полный день'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': None,
 'description': '<p><strong>PE Specialist will be working with Data Scientists to solve complex Petroleum Engineering problems by development of digital solutions and automation techniques.</strong></p> <p><strong>Responsibilities for this position may include but are not limited to:</strong></p> <ul> <li> <p>Performing Inflow Performance Relationship (IPR) analysis;</p> </li> <li> <

---

### <font color="orange"><i>Сложная часть</i></font>

 ---
### Задача 1.   

Исследуем профессию Data Scientist. Найдите как можно больше вакансий по этой профессии в Москве. Учтите, что имеет смысл искать также по другим ключевым словам, например, `аналитик данных`.

In [ ]:
import time
from scipy import stats as sts

In [ ]:
def get_one_page(params, page_number):
    params["page"] = page_number;

    req = requests.get(URL, params)
    data = json.loads(req.content.decode())

    return data;

In [ ]:
synonyms = ["аналитик данных", "data engineer", "Machine Learning Engineer", "Статистика", "анализ данных", "Data Scientist"]

URL = "https://api.hh.ru/vacancies"
main_df = df;
set_df = False;
for vacancy in synonyms:
    params = {
        "text": vacancy,
        "area": 1,
        "page": 0,
        "per_page": 100,
    }

    data = get_one_page(params, 0);
    print(data["pages"])
    print(f"Found:{data['found']}");
    number_of_pages = data["pages"];

    data = pd.json_normalize(data["items"])
    main_df = pd.concat([main_df, data], axis=0);
    for i in range(1, number_of_pages):
        print(f"Page:{i}")
        data = get_one_page(params, i);
        data = pd.json_normalize(data["items"])
        main_df = pd.concat([main_df, data], axis=0);

        time.sleep(round(sts.uniform(loc=1, scale=2).rvs(1)[0], 3));

20
Found:9065
Page:1
Page:2
Page:3
Page:4
Page:5
Page:6
Page:7
Page:8
Page:9
Page:10
Page:11
Page:12
Page:13
Page:14
Page:15
Page:16
Page:17
Page:18
Page:19
10
Found:908
Page:1
Page:2
Page:3
Page:4
Page:5
Page:6
Page:7
Page:8
Page:9
2
Found:111
Page:1
20
Found:3532
Page:1
Page:2
Page:3
Page:4
Page:5
Page:6
Page:7
Page:8
Page:9
Page:10
Page:11
Page:12
Page:13
Page:14
Page:15
Page:16
Page:17
Page:18
Page:19
20
Found:14669
Page:1
Page:2
Page:3
Page:4
Page:5
Page:6
Page:7
Page:8
Page:9
Page:10
Page:11
Page:12
Page:13
Page:14
Page:15
Page:16
Page:17
Page:18
Page:19
4
Found:359
Page:1
Page:2
Page:3


In [ ]:
print(len(main_df.index));
print(main_df.head(10))

7388
         id  premium                                             name  \
0  98612984    False                        Petroleum Engineer (Baku)   
1  98594437    False                                   Data Scientist   
2  97821551    False                               Chief data officer   
3  98592181    False            Аналитик данных/Junior DATA SCIENTIST   
4  97615359    False  Junior Data Scientist / Младший аналитик данных   
5  98467232    False                            Junior data scientist   
6  97153868    False           Junior Data Scientist (Ranking&Search)   
7  98621186    False                                   Data Scientist   
8  98562382    False                                   Data Scientist   
9  97099282    False           Стажер Java разработчик, Верхний Поиск   

   department  has_test  response_letter_required salary  address  \
0         NaN     False                     False    NaN      NaN   
1         NaN     False                     False    

В полученную выборку некоторые вакансии могли попасть несколько раз. Удалите дубликаты.

In [ ]:
main_df = main_df.drop_duplicates(subset=['id'])
main_df = main_df.reset_index();
print(len(main_df.index))

5779


Загрузите подробное описание каждой вакансии и создайте удобную для дальнейших действий таблицу данных.

In [ ]:
additional_data = None;
df_set = False;
for i in range(len(main_df.index)):
    vacancy = main_df["id"].iloc[i]
    vacancy_url = f"https://api.hh.ru/vacancies/{vacancy}"

    req = requests.get(vacancy_url)
    vacancy_info = json.loads(req.content.decode())
    vacancy_info = pd.json_normalize(vacancy_info)

    if not df_set:
        additional_data = vacancy_info;
        df_set = True;
    else:
        additional_data = pd.concat([additional_data, vacancy_info], axis=0);
    if i % 10 == 0:
        time.sleep(round(sts.uniform(loc=1, scale=1).rvs(1)[0], 3));
    if i % 10 == 0:
        print(i);

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [ ]:
additional_data

,id,premium,relations,name,insider_interview,response_letter_required,salary,address,allow_messages,department,...,test.required,employer.logo_urls,department.id,department.name,vacancy_constructor_template.bottom_picture.height,vacancy_constructor_template.bottom_picture.width,vacancy_constructor_template.bottom_picture.path,vacancy_constructor_template.bottom_picture.blurred_path,insider_interview.id,insider_interview.url
0,98612984,False,[],Petroleum Engineer (Baku),None,False,None,None,True,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,98594437,False,[],Data Scientist,None,False,NaN,NaN,True,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,97821551,False,[],Chief data officer,None,False,None,NaN,True,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,98592181,False,[],Аналитик данных/Junior DATA SCIENTIST,None,False,None,NaN,True,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,97615359,False,[],Junior Data Scientist / Младший аналитик данных,None,False,None,None,True,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,97825857,False,[],Data Scientist (CVM),None,False,None,None,True,NaN,...,NaN,NaN,bil-4934-it,"билайн: ИТ, Data, Digital",NaN,NaN,NaN,NaN,NaN,NaN
0,93943297,False,[],Менеджер по продажам курсов в сфере IT и Digit...,None,False,NaN,NaN,True,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,94697153,False,[],Senior Data Scientist,None,False,None,NaN,True,NaN,...,NaN,NaN,3529-3529-it,Сбер. IT,NaN,NaN,NaN,NaN,NaN,NaN
0,94604984,False,[],Senior Data Scientist в Trust & Safety,None,False,None,NaN,True,NaN,...,NaN,NaN,hh-1455-ds,HeadHunter::Analytics/Data Science,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def Intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

In [ ]:
columns_list1 = main_df.columns.tolist()
columns_list2 = additional_data.columns.tolist();
intersection = list(Intersection(columns_list1, columns_list2) )
print(intersection);
main_df_tmp = main_df.copy(deep=True)
additional_tmp = additional_data.copy(deep=True)
main_df_tmp = main_df_tmp.drop(intersection, axis=1)

main_df_tmp.reset_index(drop=True, inplace=True)
additional_tmp.reset_index(drop=True, inplace=True)
print(list(Intersection(main_df_tmp.columns.tolist(), columns_list2)))
main_df_tmp = pd.concat([main_df_tmp, additional_tmp], axis=1);
main_df_tmp.to_excel("data.xlsx")

['relations', 'schedule.id', 'employer.alternate_url', 'employer.url', 'id', 'working_time_intervals', 'type.name', 'insider_interview', 'alternate_url', 'experience.name', 'employer.logo_urls.240', 'salary.to', 'insider_interview.url', 'address.building', 'area.url', 'employment.id', 'address.metro.line_name', 'address.city', 'archived', 'address.metro.line_id', 'employer.logo_urls.original', 'department.id', 'address.metro_stations', 'insider_interview.id', 'experience.id', 'schedule.name', 'address.metro.station_name', 'employer.vacancies_url', 'address.lng', 'address.raw', 'working_time_modes', 'department.name', 'apply_alternate_url', 'working_days', 'employer.name', 'accept_temporary', 'department', 'salary.currency', 'address.metro.lng', 'premium', 'address.metro', 'address.lat', 'created_at', 'area.id', 'response_letter_required', 'address', 'employer.accredited_it_employer', 'salary', 'has_test', 'employer.id', 'address.description', 'address.street', 'contacts', 'employer.tru

**Полученную таблицу необходимо сохранить в формате xlsx и отправить боту вместе с решением.**

**Вопрос 1.** Сколько сейчас доступно вакансий по вашему запросу?

In [2]:
data = pd.read_excel('data.xlsx')

In [ ]:
print(f"Количество собранных вакансий: {len(data.index)}")

Количество собранных вакансий: 5779


**Вопрос 2.** Какие навыки чаще всего встречаются в вакансиях по данной специальности?

Для этого найдите соответствующее поле в данных из полного описания вакансий, проанализируйте его и составьте список навыков и количество упоминаний каждого. Визуализируйте полученную информацию по топ-15 навыков.

In [3]:
def RemoveWords(string, words):
    for word in words:
        string = string.replace(word, "");
    return string;

In [22]:
import itertools
!pip install fasttext
import fasttext

detect_language = fasttext.load_model('./models/lid.176.ftz')

In [4]:
def DetectLang(string):
    return detect_language.predict(string, k=1)[0][0].replace("__label__", "");

In [5]:
def FilterStopWords(word):
    if word in stop_words:
        return False;
    else:
        return True;

In [6]:
#fasttext embeddings for my dataset that lay in one emb space

en_dict = pd.read_csv("en_embed.csv");
ru_dict = pd.read_csv("ru_embed.csv");

In [7]:
en_dict = en_dict.set_index("word.1")
ru_dict = ru_dict.set_index("word.1")

global_dict = pd.concat([en_dict, ru_dict], axis=0);

In [17]:

# If you want to keep only the first occurrence of each duplicate index value, you can use:
#global_dict = global_dict.reset_index()
    # If you want to keep only the first occurrence of each duplicate index value, you can use:
global_dict = global_dict.drop_duplicates(subset='word.1', keep='first')
global_dict = global_dict.set_index("word.1");
global_dict = global_dict.drop(columns=["level_0"]);
print(global_dict)

                      index    word       1       2       3       4       5  \
word.1                                                                        
pe                        0 -0.0162  0.0249  0.0018  0.0262 -0.0042  0.0565   
specialist                1  0.0108 -0.0064 -0.0071  0.0239 -0.0423  0.0113   
working                   2  0.0752 -0.0083 -0.1308 -0.0180 -0.0999  0.0400   
data                      3 -0.0444 -0.0142  0.0231  0.0618  0.0051  0.0651   
scientists                4 -0.0449  0.0738 -0.0340  0.0231 -0.0442  0.0043   
...                     ...     ...     ...     ...     ...     ...     ...   
конструктивным        51191 -0.0858  0.0185  0.0312  0.0349  0.0515 -0.0243   
времяпрепровождением  51192  0.1067  0.0846  0.0723 -0.0601  0.0777  0.0163   
стрессом              51193  0.0345 -0.0033  0.0004  0.0198 -0.0198  0.0367   
таксономию            51194 -0.0637  0.0594  0.0293  0.0282 -0.0054  0.0121   
рекомендуемую         51195  0.0101  0.0402  0.0010 

In [34]:
global_dict = global_dict.drop(columns=["index"])

In [35]:
word_to_vec = {};
available_words = global_dict.index.to_list();

for word in available_words:
  embed = global_dict.loc[word];
  word_to_vec[word] = embed.to_list();
  #print(embed.to_list())

In [36]:
print(len(word_to_vec["engineering"]))
print(len(word_to_vec))

300
49366


In [63]:
arr_of_arrs = [];

for i in range(len(data.index)):
  arr_of_arrs.append(data.iloc[i]["description"].split("</p>"))

punctuation = punctuation.replace(",", "").replace(";", "");

to_remove = ["<p>", "<strong>", "</p>", "</strong>", "<ul>", "<li>", "</li>", "</ul>"];
to_remove = to_remove + [*punctuation];

#words_and_occ = {};
results = [];
used_sentences = [];

for sentences in arr_of_arrs:
  for sentence in sentences:
    sentence = RemoveWords(sentence, to_remove).strip();
    if sentence == None:
        continue;
    #lang = DetectLang(sentence);
    #print(lang)
    arr = sentence.split(";")
    arr = [item.strip().lower() for item in arr];
    arr = list(filter(None, arr))

    #split by comma
    arr = [item.split(",") for item in arr];
    #turn 2d list to 1d list
    arr = list(itertools.chain(*arr))

    #clean whitespaces
    arr = [item.strip() for item in arr]

    #split sentences into words
    arr = [item.split(" ") for item in arr];
    #remove stopwords
    arr = list(filter(FilterStopWords, list(itertools.chain(*arr))));

    avg_vec = np.zeros(300);
    i = 0;
    for word in arr:
      try:
        avg_vec += np.array(word_to_vec[word]);
        i = i + 1;
      except KeyError:
        i = i
    if i != 0:
      avg_vec /= i
      results.append(avg_vec);
      used_sentences.append(sentence)

In [64]:
results = np.array(results);
print(results)

[[-0.01336     0.0097     -0.0327     ...  0.07862667  0.04661333
  -0.01862   ]
 [-0.04866     0.00254    -0.01826    ...  0.02242     0.04854
   0.02362   ]
 [-0.02183333 -0.02615    -0.04983333 ...  0.07683333  0.03076667
  -0.03061667]
 ...
 [ 0.031325    0.061625    0.049675   ... -0.07625    -0.010725
  -0.006375  ]
 [-0.00384324  0.0319973   0.03148649 ... -0.04217297 -0.02094189
   0.00925   ]
 [-0.01866     0.0147      0.05238    ... -0.08504    -0.03062
   0.00682   ]]


In [65]:
from sklearn.decomposition import PCA

pca = PCA(n_components=20);
pca.fit(results)

PCA(n_components=20)

In [66]:
embed_to_sentence = {};

results_transformed = pca.transform(results);
for i in range(results_transformed.shape[0]):
  embed_to_sentence[results_transformed[i].sum()] = used_sentences[i]

In [68]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=15, random_state=0, n_init="auto").fit(results_transformed);

In [73]:
print(list(embed_to_sentence.keys())[:100])

[0.1187486282719428, 0.2993877195652035, -0.028930718715685123, 0.049027067322761485, -0.07552912596237214, 0.09367239710188512, 0.059596734436025534, 0.015105014010286619, 0.17227691024020408, -0.057207130728952615, 0.14175358734741017, 0.1453873280343972, -0.013846094102637412, 0.39863553721320005, -0.08031945102543708, 0.028045598206086972, 0.05868050935606164, 0.3779686225928551, 0.12149371501189549, 0.038765949328035954, 0.0779731682045634, -0.014018087007222124, 0.1328695009437037, 0.2131711711666764, -0.11342343843167503, -0.03225310730963883, 0.12793334535878484, -0.15143114048470782, -0.021371695290362216, -0.2636787560123072, -0.33622468838346364, -0.20776384734564696, 0.5152613405349007, 0.705275583682337, 0.28436976348231385, 0.08507035932780813, -0.009667883710968394, -0.018084225931340035, 0.06757641691312698, -0.14674829585554777, 0.0544788733425445, 0.03067832252261331, 1.232456182417272, 0.08485427291398259, -0.09240766114027958, 0.22692759542246724, 0.0346713098366896

[]


**Вопрос 3.** Какую зарплату готовы платить работодатели? Соберите некоторым образом статистику и постройте гистограмму.

При работе с данными о заработной плате обратите внимание на валюту и <a target="_blank" href="https://yandex.ru/search/?text=gross%2Fnet&clid=1955455&win=446&lr=213&noreask=1">gross/net</a>. Постоянно подгружать курс валюты не требуется, достаточно фиксировать какой-то один более менее актуальный.

**Вопрос 4.** Какой формат работы предлагается (в офисе / удаленно / ...)?

Проведите аналогичный анализ для наиболее привлекательной для вас профессии в любом регионе. Если это Data Scientist, то для анализа выберите другую. В данном пункте спокойно можно сделать копипасту кода.

Не забывайте про выводы.


---

### <font color="green"><i>Факультативная часть</i></font>

---

### Задача 2.   

Для *одной* из рассмотренных ранее профессий исследуйте, в каком районе Москвы данная вакансия пользуется наибольшим и наименьшим спросом.
Не забудьте про визуализацию, в частности, постройте распределений вакансий на карте.

*Напоминание.* Работа в некоторой степени творческая, поэтому конкретные детали решения в основном остаются на усмотрение автора решения.


Какие выводы можно сделать из построенных графиков?

---

### Задача 3.   

**1.** Проверьте, во скольких вакансиях среди выгруженных указана зарплата?


На основе описания вакансий с известной зарплатой попробуйте оценить ожидаемую зарплату для всех остальных вакансий. Для этого на основе текстового описания вакансии можно построить эмбеддинги (любые подходящие нейросетевые или просто one-hot кодирование). По этим эмбеддингам обучите некоторую модель предсказывать зарплату.

Не забудьте по общий пайплайн ML-моделей, а также про применимость различных методов.

**2.** Выполните кластеризацию вакансий, используя построенные эмбеддинги. Визуализируйте результаты и проинтерпретируйте кластеры.

Не забывайте про выводы.